# Sentiment Analysis on IMDb Reviews

In [1]:
from __future__ import print_function, division

Download and extract the IMDB sentiment dataset:
    
    wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    tar xfz aclImdb_v1.tar.gz

In [2]:
# Vocabulary: All words used, starting by the most frequent
with open('aclImdb/imdb.vocab') as f:
    vocab = [word.rstrip() for word in f]
    # Keep only most frequent 5000 words rather than all 90000
    # Just saving memory - the long tail occurs too few times
    # for the model to learn anything anyway
    vocab = vocab[:5000]
    print('%d words in vocabulary' % (len(vocab),))

5000 words in vocabulary


In [3]:
import re

def text_tokens(text):
    text = text.lower()
    text = re.sub("\\s", " ", text)
    text = re.sub("[^a-zA-Z' ]", "", text)
    tokens = text.split(' ')
    return tokens

In [4]:
import os

def load_dataset(dirname):
    X, y = [], []
    # Review files: neg/0_3.txt neg/10000_4.txt neg/10001_4.txt ...
    for y_val, y_label in enumerate(['neg', 'pos']):
        y_dir = os.path.join(dirname, y_label)
        for fname in os.listdir(y_dir):
            fpath = os.path.join(y_dir, fname)
            print('\r' + fpath + '   ', end='')
            with open(fpath) as f:
                tokens = text_tokens(f.read())
            X.append(tokens)
            y.append(y_val)  # 0 for 'neg', 1 for 'pos'
    print()
    return X, y

In [5]:
X_train, y_train = load_dataset('aclImdb/train/')
X_val, y_val = load_dataset('aclImdb/test/')

aclImdb/train/pos/437_9.txt      
aclImdb/test/pos/1917_10.txt    


In [10]:
len(X_train), len(X_val)

(25000, 25000)

## Bag-of-words Linear Model

In [5]:
from keras.layers import Activation, Dense, Input
from keras.models import Model
import numpy as np

class BOWSentimentModel(object):
    def __init__(self):
        bow = Input(shape=(len(vocab),), name='bow_input')
        # weights of all inputs
        sentiment = Dense(1)(bow)
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        self.model = Model(input=bow, output=sentiment)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def review_vector(self, tokens):
        vector = [0] * len(vocab)
        for t in tokens:
            try:
                vector[vocab.index(t)] = 1
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X])
        X_val = np.array([self.review_vector(r) for r in X_val])
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=25, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X])
        return self.model.predict(X)

Using TensorFlow backend.


In [7]:
sentiment = BOWSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("bo...)`
  del sys.path[0]


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/25
25000/25000 [==============================] - 5s 183us/step - loss: 0.4518 - acc: 0.8340 - val_loss: 0.3623 - val_acc: 0.8706
Epoch 2/25
25000/25000 [==============================] - 2s 97us/step - loss: 0.3133 - acc: 0.8862 - val_loss: 0.3168 - val_acc: 0.8791
Epoch 3/25
25000/25000 [==============================] - 3s 105us/step - loss: 0.2726 - acc: 0.9002 - val_loss: 0.3014 - val_acc: 0.8802
Epoch 4/25
25000/25000 [==============================] - 3s 106us/step - loss: 0.2501 - acc: 0.9070 - val_loss: 0.2939 - val_acc: 0.8812
Epoch 5/25
25000/25000 [==============================] - 3s 107us/step - loss: 0.2348 - acc: 0.9129 - val_loss: 0.2924 - val_acc: 0.8809
Epoch 6/25
25000/25000 [==============================] - 2s 97us/step - loss: 0.2230 - acc: 0.9180 - val_loss: 0.2918 - val_acc: 0.8813
Epoch 7/25
25000/25000 [==============================] - 5s 217us/step - loss: 0.2142 - acc: 0.92

## Bag-of-Words with Hidden Layer

In [6]:
from keras.layers import Activation, Dense, Input
from keras.models import Model
import numpy as np

class BOWHiddenSentimentModel(object):
    def __init__(self, N=64):
        bow = Input(shape=(len(vocab),), name='bow_input')
        # weights of all inputs
        hidden = Dense(N, activation='tanh')(bow)
        sentiment = Dense(1)(hidden)
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        self.model = Model(input=bow, output=sentiment)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def review_vector(self, tokens):
        vector = [0] * len(vocab)
        for t in tokens:
            try:
                vector[vocab.index(t)] = 1
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X])
        X_val = np.array([self.review_vector(r) for r in X_val])
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=25, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X])
        return self.model.predict(X)

Using TensorFlow backend.


In [7]:
sentiment = BOWHiddenSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("bo...)`
  


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/25
25000/25000 [==============================] - 16s 645us/step - loss: 0.3276 - acc: 0.8598 - val_loss: 0.3017 - val_acc: 0.8728
Epoch 2/25
25000/25000 [==============================] - 13s 502us/step - loss: 0.2335 - acc: 0.9052 - val_loss: 0.3119 - val_acc: 0.8682
Epoch 3/25
25000/25000 [==============================] - 15s 593us/step - loss: 0.2025 - acc: 0.9163 - val_loss: 0.3262 - val_acc: 0.8680
Epoch 4/25
25000/25000 [==============================] - 13s 529us/step - loss: 0.1772 - acc: 0.9297 - val_loss: 0.3350 - val_acc: 0.8619
Epoch 5/25
25000/25000 [==============================] - 13s 508us/step - loss: 0.1503 - acc: 0.9397 - val_loss: 0.3674 - val_acc: 0.8601
Epoch 6/25
25000/25000 [==============================] - 12s 469us/step - loss: 0.1219 - acc: 0.9526 - val_loss: 0.4186 - val_acc: 0.8573
Epoch 7/25
25000/25000 [==============================] - 14s 564us/step - loss: 0.0914 - 

## GloVe-based Model (let's start with averaging)

### Loading GloVe vocabulary

In [119]:
EMBEDDING_DIM = 50

In [120]:
# let's create a dictionary of each word in the pre-trained GloVe embeddings, saving its location indexes 
import os
GLOVE_DIR = "." # ./glove.6B/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.%dd.txt' % EMBEDDING_DIM))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [121]:
embedding_matrix = np.zeros((len(vocab) + 1, EMBEDDING_DIM))
for i, word in enumerate(vocab):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        # also, [0] is reserved for padding
        embedding_matrix[i + 1] = embedding_vector

Checking how many words have no pre-trained GloVe word embeddings:

In [20]:
1. * np.count_nonzero(np.all(embedding_matrix == 0, axis=1)) / len(vocab)  # OOV portion

0.0076

Representing reviews as sequences: What's a good compromise for sequence length?

In [40]:
lengths = sorted([len(X) for X in X_train])
percentile = 0.66
seq_cutoff = lengths[int(len(lengths)*percentile)]
print('Average: %f, Median: %d, %d%% percentile: %d tokens' % (np.mean(lengths), lengths[int(len(lengths)*0.5)], percentile*100, seq_cutoff))

Average: 233.778560, Median: 174, 66% percentile: 232 tokens


### GloVe averaging model

Questions:
  * Predict sentiment from mean embedding, or mean sentiment from each embedding?
  * `trainable=True`?
  * Projection to a "sentiment predictive" space first?

In [48]:
from keras.layers import Activation, GlobalAveragePooling1D, Dense, Embedding, Input
from keras.models import Model
import numpy as np

class GloveAvgSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64):
        self.seq_len = seq_len
        
        self.model = self.create(N)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        avg_embedded = GlobalAveragePooling1D()(seq_embedded)
        sentiment = Dense(1)(avg_embedded)
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=25, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [49]:
sentiment = GloveAvgSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/25
25000/25000 [==============================] - 2s 97us/step - loss: 0.6759 - acc: 0.6161 - val_loss: 0.6648 - val_acc: 0.6427
Epoch 2/25
25000/25000 [==============================] - 2s 93us/step - loss: 0.6538 - acc: 0.6610 - val_loss: 0.6478 - val_acc: 0.6616
Epoch 3/25
25000/25000 [==============================] - 2s 97us/step - loss: 0.6387 - acc: 0.6716 - val_loss: 0.6361 - val_acc: 0.6678
Epoch 4/25
25000/25000 [==============================] - 2s 95us/step - loss: 0.6275 - acc: 0.6806 - val_loss: 0.6263 - val_acc: 0.6758
Epoch 5/25
25000/25000 [==============================] - 3s 100us/step - loss: 0.6183 - acc: 0.6862 - val_loss: 0.6183 - val_acc: 0.6820
Epoch 6/25
25000/25000 [==============================] - 3s 102us/step - loss: 0.6106 - acc: 0.6921 - val_loss: 0.6113 - val_acc: 0.6887
Epoch 7/25
25000/25000 [==============================] - 2s 96us/step - loss: 0.6040 - acc: 0.6973 

In [50]:
from keras.layers import Activation, GlobalAveragePooling1D, Dense, Embedding, Input
from keras.models import Model
import numpy as np

class GloveSentimentAvgModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64):
        self.seq_len = seq_len
        
        self.model = self.create(N)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_sentiment = Dense(1)(seq_embedded)
        # normalize to [0, 1] range
        seq_sentiment = Activation('sigmoid')(seq_sentiment)
        sentiment = GlobalAveragePooling1D()(seq_sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=25, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [51]:
sentiment = GloveSentimentAvgModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("gl..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/25
25000/25000 [==============================] - 4s 141us/step - loss: 0.6811 - acc: 0.6116 - val_loss: 0.6736 - val_acc: 0.6403
Epoch 2/25
25000/25000 [==============================] - 3s 135us/step - loss: 0.6686 - acc: 0.6583 - val_loss: 0.6673 - val_acc: 0.6573
Epoch 3/25
25000/25000 [==============================] - 3s 129us/step - loss: 0.6641 - acc: 0.6653 - val_loss: 0.6643 - val_acc: 0.6594
Epoch 4/25
25000/25000 [==============================] - 3s 138us/step - loss: 0.6616 - acc: 0.6654 - val_loss: 0.6626 - val_acc: 0.6616
Epoch 5/25
25000/25000 [==============================] - 3s 129us/step - loss: 0.6600 - acc: 0.6693 - val_loss: 0.6620 - val_acc: 0.6580
Epoch 6/25
25000/25000 [==============================] - 4s 141us/step - loss: 0.6591 - acc: 0.6726 - val_loss: 0.6605 - val_acc: 0.6642
Epoch 7/25
25000/25000 [==============================] - 3s 124us/step - loss: 0.6582 - acc: 0.

In [55]:
from keras.layers import Activation, GlobalMaxPooling1D, Dense, Embedding, Input
from keras.models import Model
import numpy as np

class GloveHiddenMaxSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64):
        self.seq_len = seq_len
        
        self.model = self.create(N)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(seq_embedded)
        max_hidden = GlobalMaxPooling1D()(seq_hidden)
        sentiment = Dense(1)(max_hidden)
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=25, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [56]:
sentiment = GloveHiddenMaxSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/25
25000/25000 [==============================] - 32s 1ms/step - loss: 0.5609 - acc: 0.7220 - val_loss: 0.4680 - val_acc: 0.7864
Epoch 2/25
25000/25000 [==============================] - 33s 1ms/step - loss: 0.4403 - acc: 0.8014 - val_loss: 0.4283 - val_acc: 0.8055
Epoch 3/25
25000/25000 [==============================] - 33s 1ms/step - loss: 0.4051 - acc: 0.8201 - val_loss: 0.4094 - val_acc: 0.8149
Epoch 4/25
25000/25000 [==============================] - 29s 1ms/step - loss: 0.3860 - acc: 0.8296 - val_loss: 0.4032 - val_acc: 0.8171
Epoch 5/25
25000/25000 [==============================] - 29s 1ms/step - loss: 0.3715 - acc: 0.8369 - val_loss: 0.3951 - val_acc: 0.8224
Epoch 6/25
25000/25000 [==============================] - 28s 1ms/step - loss: 0.3588 - acc: 0.8412 - val_loss: 0.3899 - val_acc: 0.8248
Epoch 7/25
25000/25000 [==============================] - 31s 1ms/step - loss: 0.3507 - acc: 0.8458 - 

KeyboardInterrupt: 

In [58]:
from keras.layers import Activation, Dense, Embedding, GRU, Input
from keras.models import Model
import numpy as np

class GloveHiddenGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64):
        self.seq_len = seq_len
        
        self.model = self.create(N)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(seq_embedded)
        recursive_repr = GRU(N)(seq_hidden)
        sentiment = Dense(1)(recursive_repr)
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=5, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [59]:
sentiment = GloveHiddenGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 185s 7ms/step - loss: 0.6545 - acc: 0.5852 - val_loss: 0.6469 - val_acc: 0.5911
Epoch 2/5
25000/25000 [==============================] - 164s 7ms/step - loss: 0.4373 - acc: 0.7940 - val_loss: 0.4233 - val_acc: 0.8076
Epoch 3/5
25000/25000 [==============================] - 186s 7ms/step - loss: 0.3748 - acc: 0.8348 - val_loss: 0.3654 - val_acc: 0.8360
Epoch 4/5
25000/25000 [==============================] - 173s 7ms/step - loss: 0.3534 - acc: 0.8432 - val_loss: 0.3453 - val_acc: 0.8469
Epoch 5/5
25000/25000 [==============================] - 171s 7ms/step - loss: 0.3302 - acc: 0.8550 - val_loss: 0.3289 - val_acc: 0.8518
Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up amo

In [71]:
from keras.layers import Activation, Conv1D, Dense, Embedding, GlobalMaxPooling1D, Input
from keras.models import Model
import numpy as np

class GloveHiddenCNNSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(seq_embedded)
        seq_conv = Conv1D(N, size)(seq_hidden)
        max_conv = GlobalMaxPooling1D()(seq_conv)
        sentiment = Dense(1)(max_conv)
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=10, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [72]:
sentiment = GloveHiddenCNNSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 63s 3ms/step - loss: 0.5234 - acc: 0.7374 - val_loss: 0.4291 - val_acc: 0.8026
Epoch 2/10
25000/25000 [==============================] - 59s 2ms/step - loss: 0.4384 - acc: 0.7962 - val_loss: 0.4023 - val_acc: 0.8175
Epoch 3/10
25000/25000 [==============================] - 60s 2ms/step - loss: 0.4104 - acc: 0.8142 - val_loss: 0.3827 - val_acc: 0.8284
Epoch 4/10
25000/25000 [==============================] - 59s 2ms/step - loss: 0.3956 - acc: 0.8207 - val_loss: 0.3894 - val_acc: 0.8242
Epoch 5/10
25000/25000 [==============================] - 59s 2ms/step - loss: 0.3791 - acc: 0.8314 - val_loss: 0.3641 - val_acc: 0.8367
Epoch 6/10
25000/25000 [==============================] - 59s 2ms/step - loss: 0.3709 - acc: 0.8348 - val_loss: 0.3510 - val_acc: 0.8427
Epoch 7/10
25000/25000 [==============================] - 59s 2ms/step - loss: 0.3615 - acc: 0.8404 - 

In [77]:
from keras.layers import Activation, Conv1D, Dense, Embedding, GRU, Input
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = seq_embedded  # Dense(N, activation='tanh')(seq_embedded)
        seq_conv = Conv1D(N, size, activation='tanh')(seq_hidden)
        recursive_repr = GRU(N)(seq_conv)
        sentiment = Dense(1)(recursive_repr)
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=10, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [ ]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

In [122]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, GRU, Input, concatenate, add
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(Dropout(0.2)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh', padding='same')(Dropout(0.2)(seq_hidden))
        seq_hidden = add([seq_hidden, seq_conv])
        recursive_repr = GRU(N)(Dropout(0.2)(seq_hidden))
        sentiment = Dense(1)(Dropout(0.2)(recursive_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [123]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 256s 10ms/step - loss: 0.6920 - acc: 0.5236 - val_loss: 0.6781 - val_acc: 0.5440
Epoch 2/15
25000/25000 [==============================] - 228s 9ms/step - loss: 0.5711 - acc: 0.6836 - val_loss: 0.4038 - val_acc: 0.8200
Epoch 3/15
25000/25000 [==============================] - 198s 8ms/step - loss: 0.4324 - acc: 0.7999 - val_loss: 0.3726 - val_acc: 0.8324
Epoch 4/15
25000/25000 [==============================] - 197s 8ms/step - loss: 0.4073 - acc: 0.8129 - val_loss: 0.3636 - val_acc: 0.8359
Epoch 5/15
25000/25000 [==============================] - 198s 8ms/step - loss: 0.3950 - acc: 0.8211 - val_loss: 0.3529 - val_acc: 0.8409
Epoch 6/15
25000/25000 [==============================] - 190s 8ms/step - loss: 0.3844 - acc: 0.8274 - val_loss: 0.3483 - val_acc: 0.8474
Epoch 7/15
25000/25000 [==============================] - 201s 8ms/step - loss: 0.3754 - acc: 0

In [124]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, GRU, Input, concatenate, add
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=96, size=5):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(Dropout(0.2)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh', padding='same')(Dropout(0.2)(seq_hidden))
        seq_hidden = add([seq_hidden, seq_conv])
        recursive_repr = GRU(N)(Dropout(0.2)(seq_hidden))
        sentiment = Dense(1)(Dropout(0.2)(recursive_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [125]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 358s 14ms/step - loss: 0.6742 - acc: 0.5536 - val_loss: 0.5806 - val_acc: 0.7067
Epoch 2/15
25000/25000 [==============================] - 339s 14ms/step - loss: 0.4957 - acc: 0.7601 - val_loss: 0.4023 - val_acc: 0.8146
Epoch 3/15
25000/25000 [==============================] - 300s 12ms/step - loss: 0.4336 - acc: 0.8006 - val_loss: 0.3885 - val_acc: 0.8228
Epoch 4/15
25000/25000 [==============================] - 325s 13ms/step - loss: 0.4143 - acc: 0.8102 - val_loss: 0.3790 - val_acc: 0.8273
Epoch 5/15
25000/25000 [==============================] - 343s 14ms/step - loss: 0.3990 - acc: 0.8172 - val_loss: 0.3682 - val_acc: 0.8340
Epoch 6/15
25000/25000 [==============================] - 346s 14ms/step - loss: 0.3876 - acc: 0.8240 - val_loss: 0.3455 - val_acc: 0.8453
Epoch 7/15
25000/25000 [==============================] - 336s 13ms/step - loss: 0.3764 - 

In [79]:
EMBEDDING_DIM = 300

In [80]:
# let's create a dictionary of each word in the pre-trained GloVe embeddings, saving its location indexes 
import os
GLOVE_DIR = "." # ./glove.6B/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.%dd.txt' % EMBEDDING_DIM))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [81]:
embedding_matrix = np.zeros((len(vocab) + 1, EMBEDDING_DIM))
for i, word in enumerate(vocab):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        # also, [0] is reserved for padding
        embedding_matrix[i + 1] = embedding_vector

In [97]:
from keras.layers import Activation, Conv1D, Dense, Embedding, GRU, Input
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = seq_embedded  # Dense(N, activation='tanh')(seq_embedded)
        seq_conv = Conv1D(N, size, activation='tanh')(seq_hidden)
        recursive_repr = GRU(N)(seq_conv)
        sentiment = Dense(1)(recursive_repr)
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=10, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [98]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 267s 11ms/step - loss: 0.5957 - acc: 0.6341 - val_loss: 0.4103 - val_acc: 0.8148
Epoch 2/10
25000/25000 [==============================] - 258s 10ms/step - loss: 0.3466 - acc: 0.8490 - val_loss: 0.3117 - val_acc: 0.8654
Epoch 3/10
25000/25000 [==============================] - 259s 10ms/step - loss: 0.2985 - acc: 0.8742 - val_loss: 0.2999 - val_acc: 0.8692
Epoch 4/10
25000/25000 [==============================] - 261s 10ms/step - loss: 0.2601 - acc: 0.8944 - val_loss: 0.2963 - val_acc: 0.8718
Epoch 5/10
25000/25000 [==============================] - 249s 10ms/step - loss: 0.2280 - acc: 0.9089 - val_loss: 0.3158 - val_acc: 0.8690
Epoch 6/10
25000/25000 [==============================] - 264s 11ms/step - loss: 0.1960 - acc: 0.9232 - val_loss: 0.3129 - val_acc: 0.8681
Epoch 7/10
25000/25000 [==============================] - 256s 10ms/step - loss: 0.1630 - 

In [99]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GRU, Input
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(seq_embedded)
        seq_conv = Conv1D(N, size, activation='tanh')(seq_hidden)
        recursive_repr = GRU(N)(seq_conv)
        sentiment = Dense(1)(recursive_repr)
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=10, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [100]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 246s 10ms/step - loss: 0.6688 - acc: 0.5499 - val_loss: 0.4159 - val_acc: 0.8123
Epoch 2/10
25000/25000 [==============================] - 242s 10ms/step - loss: 0.3659 - acc: 0.8383 - val_loss: 0.3170 - val_acc: 0.8612
Epoch 3/10
25000/25000 [==============================] - 241s 10ms/step - loss: 0.3120 - acc: 0.8676 - val_loss: 0.3006 - val_acc: 0.8698
Epoch 4/10
25000/25000 [==============================] - 237s 9ms/step - loss: 0.2874 - acc: 0.8776 - val_loss: 0.3160 - val_acc: 0.8622
Epoch 5/10
25000/25000 [==============================] - 237s 9ms/step - loss: 0.2623 - acc: 0.8904 - val_loss: 0.2983 - val_acc: 0.8744
Epoch 6/10
25000/25000 [==============================] - 235s 9ms/step - loss: 0.2395 - acc: 0.9027 - val_loss: 0.2925 - val_acc: 0.8733
Epoch 7/10
25000/25000 [==============================] - 226s 9ms/step - loss: 0.2166 - acc:

In [101]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GRU, Input
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(Dropout(0.2)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh')(Dropout(0.2)(seq_hidden))
        recursive_repr = GRU(N)(Dropout(0.2)(seq_conv))
        sentiment = Dense(1)(Dropout(0.2)(recursive_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [102]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 219s 9ms/step - loss: 0.6853 - acc: 0.5331 - val_loss: 0.6603 - val_acc: 0.5762
Epoch 2/15
25000/25000 [==============================] - 216s 9ms/step - loss: 0.6729 - acc: 0.5844 - val_loss: 0.6708 - val_acc: 0.5466
Epoch 3/15
25000/25000 [==============================] - 209s 8ms/step - loss: 0.4896 - acc: 0.7744 - val_loss: 0.4113 - val_acc: 0.8129
Epoch 4/15
25000/25000 [==============================] - 270s 11ms/step - loss: 0.3722 - acc: 0.8356 - val_loss: 0.3208 - val_acc: 0.8614
Epoch 5/15
25000/25000 [==============================] - 274s 11ms/step - loss: 0.3432 - acc: 0.8502 - val_loss: 0.3253 - val_acc: 0.8573
Epoch 6/15
25000/25000 [==============================] - 20297s 812ms/step - loss: 0.3308 - acc: 0.8560 - val_loss: 0.3098 - val_acc: 0.8674
Epoch 7/15
25000/25000 [==============================] - 236s 9ms/step - loss: 0.3181 - a

In [103]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GRU, Input
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=True)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(Dropout(0.2)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh')(Dropout(0.2)(seq_hidden))
        recursive_repr = GRU(N)(Dropout(0.2)(seq_conv))
        sentiment = Dense(1)(Dropout(0.2)(recursive_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [104]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 236s 9ms/step - loss: 0.6853 - acc: 0.5355 - val_loss: 0.6445 - val_acc: 0.5778
Epoch 2/15
25000/25000 [==============================] - 224s 9ms/step - loss: 0.5460 - acc: 0.6940 - val_loss: 0.3461 - val_acc: 0.8540
Epoch 3/15
25000/25000 [==============================] - 222s 9ms/step - loss: 0.3095 - acc: 0.8704 - val_loss: 0.2930 - val_acc: 0.8801
Epoch 4/15
25000/25000 [==============================] - 432s 17ms/step - loss: 0.2517 - acc: 0.8979 - val_loss: 0.2939 - val_acc: 0.8775
Epoch 5/15
25000/25000 [==============================] - 261s 10ms/step - loss: 0.2213 - acc: 0.9123 - val_loss: 0.2935 - val_acc: 0.8807
Epoch 6/15
25000/25000 [==============================] - 241s 10ms/step - loss: 0.1913 - acc: 0.9268 - val_loss: 0.3349 - val_acc: 0.8721
Epoch 7/15
25000/25000 [==============================] - 233s 9ms/step - loss: 0.1745 - acc:

In [105]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GRU, Input
from keras.models import Model
from keras import regularizers
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=True,
                                 embeddings_regularizer=regularizers.l1(1e-4))(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(Dropout(0.2)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh')(Dropout(0.2)(seq_hidden))
        recursive_repr = GRU(N)(Dropout(0.2)(seq_conv))
        sentiment = Dense(1)(Dropout(0.2)(recursive_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [106]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 261s 10ms/step - loss: 12.7019 - acc: 0.5245 - val_loss: 1.9663 - val_acc: 0.5365
Epoch 2/15
25000/25000 [==============================] - 242s 10ms/step - loss: 1.1994 - acc: 0.5304 - val_loss: 0.8885 - val_acc: 0.5428
Epoch 3/15
25000/25000 [==============================] - 289s 12ms/step - loss: 0.7997 - acc: 0.6358 - val_loss: 0.6382 - val_acc: 0.7862
Epoch 4/15
25000/25000 [==============================] - 282s 11ms/step - loss: 0.6308 - acc: 0.8184 - val_loss: 0.5781 - val_acc: 0.8556
Epoch 5/15
25000/25000 [==============================] - 256s 10ms/step - loss: 0.5790 - acc: 0.8591 - val_loss: 0.5475 - val_acc: 0.8734
Epoch 6/15
25000/25000 [==============================] - 498s 20ms/step - loss: 0.5368 - acc: 0.8722 - val_loss: 0.5393 - val_acc: 0.8714
Epoch 7/15
25000/25000 [==============================] - 236s 9ms/step - loss: 0.5201 - 

In [107]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GRU, Input
from keras.models import Model
from keras import regularizers
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=True)(seq_indices)
        seq_hidden = Dense(N, activation='tanh', kernel_regularizer=regularizers.l2(1e-4))(Dropout(0.5)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh', kernel_regularizer=regularizers.l2(1e-4))(Dropout(0.5)(seq_hidden))
        recursive_repr = GRU(N)(Dropout(0.5)(seq_conv))
        sentiment = Dense(1)(Dropout(0.5)(recursive_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [108]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 283s 11ms/step - loss: 0.7116 - acc: 0.5146 - val_loss: 0.6950 - val_acc: 0.5431
Epoch 2/15
25000/25000 [==============================] - 252s 10ms/step - loss: 0.6941 - acc: 0.5506 - val_loss: 0.6810 - val_acc: 0.5692
Epoch 3/15
25000/25000 [==============================] - 241s 10ms/step - loss: 0.4502 - acc: 0.8001 - val_loss: 0.3250 - val_acc: 0.8708
Epoch 4/15
25000/25000 [==============================] - 265s 11ms/step - loss: 0.3299 - acc: 0.8701 - val_loss: 0.3002 - val_acc: 0.8806
Epoch 5/15
25000/25000 [==============================] - 280s 11ms/step - loss: 0.3039 - acc: 0.8846 - val_loss: 0.3259 - val_acc: 0.8690
Epoch 6/15
25000/25000 [==============================] - 255s 10ms/step - loss: 0.2832 - acc: 0.8934 - val_loss: 0.3089 - val_acc: 0.8752
Epoch 7/15
25000/25000 [==============================] - 245s 10ms/step - loss: 0.2677 - 

In [111]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, GRU, Input, concatenate, add
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(Dropout(0.2)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh', padding='same')(Dropout(0.2)(seq_hidden))
        recursive_repr = GRU(N)(Dropout(0.2)(seq_hidden))  # !!!!!!!!!!!!!!
        full_repr = concatenate([GlobalAveragePooling1D()(seq_hidden), GlobalMaxPooling1D()(seq_conv), recursive_repr])
        sentiment = Dense(1)(Dropout(0.2)(full_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [112]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 256s 10ms/step - loss: 0.4742 - acc: 0.7680 - val_loss: 0.3789 - val_acc: 0.8301
Epoch 2/15
25000/25000 [==============================] - 244s 10ms/step - loss: 0.3781 - acc: 0.8327 - val_loss: 0.3371 - val_acc: 0.8519
Epoch 3/15
25000/25000 [==============================] - 254s 10ms/step - loss: 0.3522 - acc: 0.8445 - val_loss: 0.3193 - val_acc: 0.8616
Epoch 4/15
25000/25000 [==============================] - 282s 11ms/step - loss: 0.3330 - acc: 0.8564 - val_loss: 0.3063 - val_acc: 0.8670
Epoch 5/15
25000/25000 [==============================] - 285s 11ms/step - loss: 0.3163 - acc: 0.8644 - val_loss: 0.2938 - val_acc: 0.8743
Epoch 6/15
25000/25000 [==============================] - 256s 10ms/step - loss: 0.3068 - acc: 0.8694 - val_loss: 0.2848 - val_acc: 0.8795
Epoch 7/15
25000/25000 [==============================] - 247s 10ms/step - loss: 0.2923 - 

In [113]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, GRU, Input, concatenate, add
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(Dropout(0.2)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh', padding='same')(Dropout(0.2)(seq_hidden))
        recursive_repr = GRU(N)(Dropout(0.2)(seq_hidden))  # !!!!!!!!!!!!!!
        full_repr = add([GlobalAveragePooling1D()(seq_hidden), GlobalMaxPooling1D()(seq_conv), recursive_repr])
        sentiment = Dense(1)(Dropout(0.2)(full_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [114]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 245s 10ms/step - loss: 0.4829 - acc: 0.7582 - val_loss: 0.3713 - val_acc: 0.8356
Epoch 2/15
25000/25000 [==============================] - 222s 9ms/step - loss: 0.3767 - acc: 0.8349 - val_loss: 0.3305 - val_acc: 0.8570
Epoch 3/15
25000/25000 [==============================] - 3878s 155ms/step - loss: 0.3403 - acc: 0.8516 - val_loss: 0.3100 - val_acc: 0.8674
Epoch 4/15
25000/25000 [==============================] - 241s 10ms/step - loss: 0.3201 - acc: 0.8606 - val_loss: 0.2905 - val_acc: 0.8754
Epoch 5/15
25000/25000 [==============================] - 240s 10ms/step - loss: 0.3039 - acc: 0.8713 - val_loss: 0.2838 - val_acc: 0.8774
Epoch 6/15
25000/25000 [==============================] - 238s 10ms/step - loss: 0.2916 - acc: 0.8758 - val_loss: 0.2759 - val_acc: 0.8840
Epoch 7/15
25000/25000 [==============================] - 236s 9ms/step - loss: 0.2828 - 

In [115]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, GRU, Input, concatenate, add
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=64, size=3):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(Dropout(0.2)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh', padding='same')(Dropout(0.2)(seq_hidden))
        seq_hidden = add([seq_hidden, seq_conv])
        recursive_repr = GRU(N)(Dropout(0.2)(seq_hidden))
        sentiment = Dense(1)(Dropout(0.2)(recursive_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [116]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 245s 10ms/step - loss: 0.6553 - acc: 0.5832 - val_loss: 0.5073 - val_acc: 0.7815
Epoch 2/15
25000/25000 [==============================] - 252s 10ms/step - loss: 0.4638 - acc: 0.7801 - val_loss: 0.3531 - val_acc: 0.8464
Epoch 3/15
25000/25000 [==============================] - 241s 10ms/step - loss: 0.3566 - acc: 0.8450 - val_loss: 0.3126 - val_acc: 0.8631
Epoch 4/15
25000/25000 [==============================] - 232s 9ms/step - loss: 0.3350 - acc: 0.8538 - val_loss: 0.3016 - val_acc: 0.8717
Epoch 5/15
25000/25000 [==============================] - 284s 11ms/step - loss: 0.3138 - acc: 0.8653 - val_loss: 0.3160 - val_acc: 0.8576
Epoch 6/15
25000/25000 [==============================] - 580s 23ms/step - loss: 0.3074 - acc: 0.8688 - val_loss: 0.2865 - val_acc: 0.8770
Epoch 7/15
25000/25000 [==============================] - 233s 9ms/step - loss: 0.2920 - ac

In [117]:
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, GRU, Input, concatenate, add
from keras.models import Model
import numpy as np

class GloveCNNGRUSentimentModel(object):
    def __init__(self, seq_len=seq_cutoff, N=96, size=5):
        self.seq_len = seq_len
        
        self.model = self.create(N, size)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
    def create(self, N, size):
        seq_indices = Input(shape=(self.seq_len,), name='seq_input')                    
        seq_embedded = Embedding(input_dim=len(vocab) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
                                 input_length=self.seq_len, trainable=False)(seq_indices)
        seq_hidden = Dense(N, activation='tanh')(Dropout(0.2)(seq_embedded))
        seq_conv = Conv1D(N, size, activation='tanh', padding='same')(Dropout(0.2)(seq_hidden))
        seq_hidden = add([seq_hidden, seq_conv])
        recursive_repr = GRU(N)(Dropout(0.2)(seq_hidden))
        sentiment = Dense(1)(Dropout(0.2)(recursive_repr))
        # normalize to [0, 1] range
        sentiment = Activation('sigmoid')(sentiment)

        return Model(input=seq_indices, output=sentiment)
        
    def review_vector(self, tokens):
        vector = [0] * self.seq_len
        if len(tokens) > self.seq_len:
            # Remove the middle
            tokens = tokens[: self.seq_len // 2] + ['SINGLE_PADDING_IN_THE_MIDDLE'] + tokens[-self.seq_len // 2 :]
        for i, t in enumerate(tokens):
            try:
                vector[i] = vocab.index(t) + 1  # reserving 0 for padding
            except:
                pass  # ignore missing words
        return vector

    def train(self, X, y, X_val, y_val):
        print('Vectorizing...')
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        X_val = np.array([self.review_vector(r) for r in X_val], dtype='int32')
        print('Fitting...')
        self.model.fit(X, y, validation_data=(X_val, y_val), epochs=15, verbose=1)

    def predict(self, X):
        X = np.array([self.review_vector(r) for r in X], dtype='int32')
        return self.model.predict(X)

In [118]:
sentiment = GloveCNNGRUSentimentModel()
sentiment.train(X_train, y_train, X_val, y_val)

test_text = 'Good story about a backwoods community in the Ozarks around the turn of the century. Moonshine is the leading industry, fighting and funning the major form of entertainment. One day a stranger enters the community and causes a shake-up among the locals. Beautiful scenery adds much to the story.'
test_tokens = text_tokens(test_text)
print(test_text, sentiment.predict([test_tokens])[0])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("se...)`


Vectorizing...
Fitting...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 352s 14ms/step - loss: 0.6499 - acc: 0.5782 - val_loss: 0.4484 - val_acc: 0.7847
Epoch 2/15
25000/25000 [==============================] - 339s 14ms/step - loss: 0.3800 - acc: 0.8325 - val_loss: 0.3208 - val_acc: 0.8596
Epoch 3/15
25000/25000 [==============================] - 1646s 66ms/step - loss: 0.3367 - acc: 0.8536 - val_loss: 0.3194 - val_acc: 0.8624
Epoch 4/15
25000/25000 [==============================] - 328s 13ms/step - loss: 0.3158 - acc: 0.8659 - val_loss: 0.3333 - val_acc: 0.8537
Epoch 5/15
25000/25000 [==============================] - 334s 13ms/step - loss: 0.2987 - acc: 0.8722 - val_loss: 0.2906 - val_acc: 0.8778
Epoch 6/15
25000/25000 [==============================] - 340s 14ms/step - loss: 0.2913 - acc: 0.8784 - val_loss: 0.2852 - val_acc: 0.8784
Epoch 7/15
25000/25000 [==============================] - 357s 14ms/step - loss: 0.2737 -

### 